# Assignment 4
*By Ryan Cox*

In [12]:
using LinearAlgebra
using Plots
using Plots.Measures
using MAT
using Statistics

plotlyjs();

## Q1

First let us load our data. We'll then plot all the seismograms together, just to demonstrate their similarity. This graph is useless for anything else, although it has a certain asthetic appeal.

In [13]:
seismograms = matread("seismograms.mat.mat")["seismograms"]
# I assume the format of the data is that each row is single seismogram.
# Each value is a displacement  and they are ordered chronoloigcally over some timescale?

"""
Plots the seismogram matrix such that all the seismograms are overlaid.
"""
function plotSeismogramMatrix(S::Matrix)::Plots.Plot
    # Setup a variable to store the plots
    p = plot(legend=false, xlabel="Time", ylabel="Displacement", title="Seismograms") # I don't know what the units are
    # Loop over all the seismograms
    for row in 1:size(S)[1]
        p = plot!(S[row,:])
    end
    return p
end

plotSeismogramMatrix(seismograms)

Now we will perform singular value decomposition on the data. Heatmap representations of V are used to illustrate its structure. Singular values are also displayed, plotted against their indicies.

In [14]:
decomposition = svd(seismograms)
U = decomposition.U
S = decomposition.S # The book calls it L. It is provided here already as a vector of the singular values.
V = decomposition.V
Vt = decomposition.Vt # transpose of V

unweighted = heatmap(Vt, title="Unweighted Vt", yflip=true, colorbar_title="Vt matrix")
weighted = heatmap(diagm(S)*Vt, title="Weighted Vt", yflip=true, colorbar_title="S*V matrix")

display(unweighted)
display(weighted)

display(plot(S, title="Singular values spectrum", ylabel="S", legend=false))
display(heatmap(U, title="Unweighted U", yflip=true, colorbar_title="U matrix"))

Now we'll plot various approximations together with the actual result. I've plotted them very large so that the differences in lines are more apparent.

In [74]:
# Just a custom type for storing U_p * S_p * V_p^T
struct ApproxMatrix
    mat::Matrix
    p::Int # First p singular values used to make mat
end

"""
Generate an approximation of the seismogram matrix using the first p singular values
"""
function approximateMatrix(U::Matrix, S::Vector, Vt::Matrix, p::Integer)::ApproxMatrix
    # Truncate component matrices
    Up = U[:,1:p]
    Sp = S[1:p] 
    Vtp = Vt[1:p,:]
    # Save as our custom type
    ApproxMatrix(Up*diagm(Sp)*Vtp, p)
end


"""
Plot a single seismogram (matrix row) with actual value and several p values.
"""
function plotSeismogram(row::Int, actualMatrix::Matrix, approxMatrices::Vector{ApproxMatrix})::Plots.Plot
    actual = actualMatrix[row,:]
    fig = plot(actual, title="Seismogram $row", xlabel="Time", ylabel="Displacement", label="Actual data", xticks=:native)
    for approx in approxMatrices
        plot!(approx.mat[row,:], label="First $(approx.p) singular values")
    end
    return fig
end


# Each element p in this list causes an approximation to be generated with the first p singular values
pValues = [1,2,3]

approxMatrices = approximateMatrix.(Ref(U), Ref(S), Ref(Vt), pValues) # refs mean only pValues is broadcast

plotRange = 1:5:30 # row indicies of the seismograms to display

figures = plotSeismogram.(plotRange, Ref(seismograms), Ref(approxMatrices))
for i in 2:6
    for j in 1:4
        figures[i][1][j][:label] = ""
    end
end
plt = plot(figures..., layout=(6, 1), size=(1500, 3000), margin=10mm)

#display(plt)

*NB: For mysterious reasons the y-axis label is being cut off, but it *should* show "Displacement" as a label.*

I decided to show the synthesis results on top of the raw data because it makes it easier to compare than an array of subplots, at least when the plot is big enough that variation is distinct. We can see that even one or two values is enough to roughly approximate the data. With three values we can get some very good matches, such as the right of seisogram 6 and some poorer matches, as in the start of seismogram 26. The approximations match peak positions fairly reliably but the peak amplutude is more commonly flawed. Large amplitude changes seem to be better approximated than small ones, so the envelopes of large variation tend to be more accurate than the surronding calmer regions.

## Q2

As the very helpful location notes explain, the trick is peturbing the model. Arrival time $d_i$ is
$$d_i = \frac{1}{\alpha} \left[ (x-x_i)^2 + (y-y_i)^2 + (z-z_i)^2 \right]^{1/2} + t$$
where $(x,y,z)$ is the position of the station, $x_i,y_i,z_i)$ is the position of the hypocentre and $t$ is a constant adjustment to the time.

We can linearise this by first peturbing it from a starting point, $d_i^o$
$$d_i = d_i^o + \sum_j \frac{\partial d_i}{\partial m_j}$$
$$\Delta d_i = d_i - d_i^o = \sum_j \frac{\partial d_i}{\partial m_j}$$
$$\Delta \vec{d} = G \Delta\vec{m} \quad \text{where } G_{ij} = \frac{\partial d_i}{\partial m_j}$$

The derivatives for the spatial parameters are of the form
$$G_{i1} = \frac{\partial d_i}{\partial m_j} = \frac{\partial d_i}{\partial x} = \frac{x-x_i}{\alpha} \left[ (x-x_i)^2 + (y-y_i)^2 + (z-z_i)^2 \right]^{-1/2}$$
and for the temporal parameter
$$G_{i4} = \frac{\partial d_i}{\partial m_4} = \frac{\partial d_i}{\partial t} = 1$$

With this we can begin our code. This time I'm going to work closely from the example code, which should prove a mildly interesting experiment in seeing how well MATLAB translates to Julia.

In [77]:
"""
Results of earthquake location function.

Attributes
    m::Vector{Float64} - Parameters found by fitting
        m[1] - x coordinate of earthquake hypocenter
        m[2] - y coordinate of earthquake hypocenter
        m[3] - z coordinate of earthquake hypocenter (should be positive)
        m[4] - original time of the earthquake
    i::Int - Number of iterations to achieve convergence
    outputMatrix::Matrix - Stores results of all iterations in form [iteration, parameters, perturbation]
"""
struct EQAnalysis
    m::Vector{Float64}
    i::Int
    outputMatrix::Matrix
end


"""
Locate an earthquake given P-wave arrival times in a uniform velocity environment.

Based on incomplete MATLAB sample code by John Townend. Translated to Julia and completed by Ryan Cox.

Parameters
    x::Vector - Seisometer locations, x coordinates
    y::Vector - Seisometer locations, y coordinates
    z::Vector - Seisometer locations, z coordinates
    tP::Vector - P-wave arrival times
    speed::Real - Speed of P-waves in medium. Defaults to six length units per time unit.

Returns EQAnalysis object
"""
function locateEQ(x::Vector, y::Vector, z::Vector, tP::Vector, speed::Real=6)::EQAnalysis
    # Prep some variables
    N = length(x) # Number of seisometers
    normdm = Inf # (squared) length of model perturbation \
    i=0 # iteration counter

    # Specify starting position
    m = [mean(x), mean(y), mean(z)+10, 0] # positive z is down and earthquakes tend to occur deeper than seisometers

    # Matrix to hold results
    #outputMatrix = [i m normdm] # iteration, parameters, perturbation
    outputMatrix = [1 1 1] # TEMP fix of borkedness

    # Prepare screen output
    println("Iteration | (x,y,z,t) | normdm sqd | error")
    println("$i | ($(m[1]), $(m[2]), $(m[3]), $(m[4])) | $normdm | ?")

    # Iteratively peturb model
    while normdm >= 1e-5 # stopping criterion
        G = generateKernel(m, x, y, z, speed) # generate data kernel such that dd=G*dm
        R = hypodistance(m, x, y, z) # distance to current hypocenter
        d = (R / speed) .- m[4] # predicted arrival time (travel time + origin time)
        # the original code has the origin time added on but subtracting it proved necessary to make it converge
        # but m[4] seems to be negative anyway so maybe that explains the sign problems
        Δd = d - tP # residual arrival time
        Gg = inv(transpose(G)*G) * transpose(G) # generalised inverse of kernal
        dm = Gg * Δd # model perturbation
        normdm = transpose(dm) * dm # (squared) length of perturbation
        m += dm # update model
        i += 1 # increment iteration counter
        err = (hypodistance(m, x, y, z)./speed .- m[4]) - tP # display error so we can see if we're getting closer
        err = transpose(err)*err
        println("$i | ($(m[1]), $(m[2]), $(m[3]), $(m[4])) | $normdm | $err")
        # outputMatrix = [outputMatrix;i m normdm] # ~~inefficent in Julia, should be improved~~ BROKEN
    end

    # Plot it all
    plt = scatter(x, y, -z, label="Seismometers") # Inverting z for more intutive render
    scatter!([m[1]], [m[2]], -[m[3]], label="EQ") # scatter wants points as arrays so we wrap parameters in square brackets
    # For each seisometer we can use (detection time - origin time) * speed to get the distance it had to travel
    # We can then plot this as a sphere around our seisometers.
    spheres!.((tP .+ m[4])*6, x, y, z) # Note the addition instead of subtraction - a consequence of the way we had to subtract dm to get convergence.
    display(plt)

    return EQAnalysis(m, i, outputMatrix)
end


"""
Calculate hypocentral distance

Parameters
    m::Vector - Model parameters (x0,y0,zo,t0)
    x::Vector - Seisometer locations, x coordinates
    y::Vector - Seisometer locations, y coordinates
    z::Vector - Seisometer locations, z coordinates

Returns distance between hypocentre and each station as Vector
"""
function hypodistance(m::Vector, x::Vector, y::Vector, z::Vector)::Vector
    dx = x .- m[1]
    dy = y .- m[2]
    dz = z .- m[3]
    R = sqrt.(dx.^2 + dy.^2 + dz.^2)
end


"""
Generates kernel G from the travel time gradient.

Parameters
    m::Vector - Model parameters (x0,y0,zo,t0)
    x::Vector - Seisometer locations, x coordinates
    y::Vector - Seisometer locations, y coordinates
    z::Vector - Seisometer locations, z coordinates
    speed::Real - Speed of P-waves in medium

Returns kernel as Matrix
"""
function generateKernel(m::Vector, x::Vector, y::Vector, z::Vector, speed::Real=6)::Matrix{Float64}
    N = length(x)
    R = hypodistance(m, x, y, z)

    G = Matrix{Float64}(undef, (N, 4))
    G[:,1] .= (x .- m[1]) ./ (speed * R)
    G[:,2] .= (y .- m[2]) ./ (speed * R)
    G[:,3] .= (z .- m[3]) ./ (speed * R)
    G[:,4] .= 1

    return G
end


"""
Fancy plotting preset.
Takes radius and origin coordinates and plots the associated sphere.
This automagically creates the sphere and sphere! functions, which take the parameters
    r::Real - Radius of sphere
    x0::Real - x coordinate of sphere origin
    y0::Real - y coordinate of sphere origin
    z0::Real - z coordinate of sphere origin
"""
@userplot Spheres
@recipe function f(h::Spheres)
    r, x0, y0, z0 = h.args # I should really include input checking but for this little project I can get away without it.

    N = 22
    θ = LinRange(0, 2π, N)
    ϕ = LinRange(0, π, N)
    x = r*cos.(ϕ) * transpose(sin.(θ)) .+ x0
    y = r*sin.(ϕ) * transpose(sin.(θ)) .+ y0
    z = repeat(r*transpose(cos.(θ)) .+ z0, outer=[N, 1])

    seriestype := :surface
    colorbar := false
    seriesalpha := 0.1
    seriescolor := :grey
    xticks := :native

    x, y, z
end

# Seismometer locations in kilometres
x = [40,20,0,40,20,0]
y = [40,40,40,0,0,0]
z = [0,0,0,0,0,0]
# The sample code doesn't include z but there is no reason we can't have seisometers distrubuted in that axis.

# O-wave arrival times in seconds
tP1=[4.16,3.36,5.83,6.68,6.27,7.75] # case 1
tP2=[12.46,15.77,19.09,13.65,16.74,19.90] # case 2
e=[0.1,-0.2,-0.3,0.2,0,0] # errors

eq1 = locateEQ(x, y, z, tP1)
eq2 = locateEQ(x, y, z, tP2)




Iteration | (x,y,z,t) | normdm sqd | error
0 | (20.0, 20.0, 10.0, 0.0) | Inf | ?
1 | (26.165, 29.881259499793643, 9.11591549683476, -1.154115805731403) | 137.76010300403811 | 0.2956128619422323
2 | (26.001289252861696, 30.01538063509899, 8.093544023869649, -0.997873501398904) | 1.114444774060156 | 6.461676187725861e-5
3 | (26.00639868308497, 30.018167108287187, 8.095992759651821, -0.9947356622383299) | 4.971305156361051e-5 | 1.6305944231931156e-5
4 | (26.006399057460587, 30.018166561031137, 8.095979911928477, -0.9947362789384726) | 1.6588396048014664e-10 | 1.630594384340562e-5


Iteration | (x,y,z,t) | normdm sqd | error
0 | (20.0, 20.0, 10.0, 0.0) | Inf | ?
1 | (48.97999999999999, 23.90954264028209, 76.0283009710682, -7.606761057162876) | 5272.724266562919 | 203.20084642850222
2 | (115.68945146151088, 34.36526206271624, 93.66058172091843, 4.30904965850239) | 5012.356852389447 | 32.18538855530855
3 | (144.09652604617625, 38.9241577659838, 47.77774702733591, 4.593328048715817) | 2933.0607502137746 | 19.433999343445194
4 | (108.35658669794012, 33.21072941599605, 9.274797025781048, -1.455853494541957) | 2829.0562842915656 | 1.717812073531086
5 | (117.69875100627169, 34.66267962117044, 9.290350838389465, 0.6050555403550391) | 93.63178133337708 | 0.0001886479516758189
6 | (118.65682176776791, 34.80805016529717, 8.868881848324587, 0.760708818553347) | 1.1408962317338256 | 9.350234692311524e-6
7 | (118.66107146519295, 34.80863718624831, 8.845335571265748, 0.7613096198610191) | 0.0005731926473443483 | 8.94361809209197e-6
8 | (118.66102579122057, 34.808629815929315, 8

EQAnalysis([118.66102579122057, 34.808629815929315, 8.84527903155976, 0.7613017393720943], 8, [1 1 1])